# Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def get_intensities():

  ###############################################
  # define intensities
  # intensity = np.sqrt([0,1,2,3,4,5,6,7])
  # intensity = np.sqrt([0,1,2,3])
  intensity = np.array([0.0253, 0.7282, 1.4643, 1.1865, 1.3226, 1.0476, 0.8985, 0.5129])
  ###############################################
  # Normalize ref constellation power to one
  pow_avg = np.sqrt(np.mean(np.abs(intensity)**2))  
  intensity_norm = intensity/pow_avg 

  return intensity_norm

def tx_PAM_Gen(syst): 
  '''
  This fuction generates PAM symbols based on the PAM order
  input: 
    syst: attribute dictionary of system parameters 
  output:    
    Tx_sym: the index of the symbols (any of [0,1,2,...,M]) where M is the PAM order
    Tx_coords: coordinates for Tx_sym
  '''   
  ###############################################
  # geterate random symbols with the size (N,1)
  Tx_syms = np.random.randint(0,syst.M,(syst.frame_size,1))

  ###############################################
  # get PAM reference coordinates
  ref_const = get_intensities()
  
  
  ###############################################
  # assign the intensities to the symbols 
  Tx_coords = ref_const[Tx_syms[:,0].astype(int)].reshape(-1,1)

  return Tx_syms, Tx_coords


def add_transmitter_noise(signal, syst, P_sig_before_PD, P_sig_after_PD):
  epsilon = 1e-20 
  noise = (np.random.randn(np.max(signal.shape))).reshape(-1,1)
  ASE_var=(1-syst.alpha)*(10**(-syst.SNR_total/10))*P_sig_after_PD
  var_n1 = ((-6*P_sig_before_PD)+np.sqrt((36*P_sig_before_PD**2)+12*ASE_var))/6
  var_n1 =  np.max([epsilon, var_n1])
  # print(var_n1)
  Data_out_ASE = signal+np.sqrt(var_n1)*noise;
  return Data_out_ASE
  
def add_thermal_noise(signal, syst, P_sig_before_PD, P_sig_after_PD):
  noise = (np.random.randn(np.max(signal.shape))).reshape(-1,1)
  Thermal_var=(syst.alpha)*(10**(-syst.SNR_total/10))*P_sig_after_PD
  Data_out=signal+np.sqrt(Thermal_var)*noise
  return Data_out
  

# Main

In [ ]:
def main(): 
  ###############################################
  # channel parameters 
  syst = AttrDict()    
  syst.M = 8
  syst.frame_size = int(1e6)
  
  for syst.alpha in np.linspace(0.9,0.1, num=1): # 10 valuses in [0,1] including 0 and 1 themselves.
    print('\n\n\t \u03B1= ', syst.alpha)
    print('\t---------------------------------')
    SER = []
    SNR = []
    for syst.SNR_total in range(23,28,1):
      Totla_S_error  = 0
      totoal_symbols = 0
      # count at least 100 symbol errors      
      while Totla_S_error<=100: 

        ###############################################
        # input symbols generation
        symbs, ref_PAM = tx_PAM_Gen(syst)


        P_sig_before_PD=np.mean(np.abs(ref_PAM)**2)
        P_sig_after_PD=np.mean(np.abs(ref_PAM)**4)
        ###############################################
        # add transmitter noise 
        data_Tx_noise = add_transmitter_noise(ref_PAM, syst, P_sig_before_PD, P_sig_after_PD)
        # data_Tx_noise = ref_PAM

        ###############################################
        # add jphoto detector effect 
        photo_detector = np.abs(data_Tx_noise)**2        

        ###############################################
        # add thermal noise 
        data_Thermal_noise = add_thermal_noise(photo_detector, syst, P_sig_before_PD, P_sig_after_PD).reshape(1,-1)
        # data_Thermal_noise = add_noise(photo_detector, syst).reshape(1,-1)
        ###############################################
        # calculate symbol errors

        ref_const = get_intensities()

        # demodulate
        RX = []
        for i in range(np.max(data_Thermal_noise.shape)):
          idx = np.argmin(np.abs(data_Thermal_noise[0,i]-ref_const**2))       
          RX.append(idx) 
        RX = np.array(RX).reshape(-1,1)

        s_errors = np.sum(symbs!= RX)
        totoal_symbols+=np.max(symbs.shape)
        Totla_S_error += s_errors



          # plt.figure(dpi=200)
          # plt.scatter(data_Thermal_noise[0,:],np.zeros_like(data_Tx_noise)[:,0],s=2, alpha=0.5)
          # plt.scatter(ref_const**2,np.zeros_like(ref_const))
          # plt.show()
          
      SER.append(Totla_S_error/totoal_symbols)    
      SNR.append(syst.SNR_total)      
      print('\t SER for SNR= {} is {}'.format(SNR[-1], SER[-1]))

    SNR = np.array(SNR)
    SER = np.array(SER)
    print('\n\t \u03B1= ', syst.alpha)    
    print('SNR range is: \n',SNR)
    print('SER is: \n', SER)
if __name__ == "__main__":  
  main()



	 α=  0.9
	---------------------------------
	 SER for SNR= 23 is 0.06819
	 SER for SNR= 24 is 0.042251
	 SER for SNR= 25 is 0.024176
	 SER for SNR= 26 is 0.012163
	 SER for SNR= 27 is 0.005316

	 α=  0.9
SNR range is: 
 [23 24 25 26 27]
SER is: 
 [0.06819  0.042251 0.024176 0.012163 0.005316]
